# Unsupervised Machine Learning for the Classification of Astrophysical X-ray Sources
###### *Víctor Samuel Pérez Díaz<sup>1,2</sup>, Rafael Martinez-Galarza<sup>1</sup>, Alexander Caicedo-Dorado<sup>2</sup>, Raffaele D'Abrusco<sup>1</sup>*

*1. Center for Astrophysics | Harvard & Smithsonian, 2. Universidad del Rosario*

Contact ```vperezdiaz@cfa.harvard.edu``` for questions or comments.


#### Classification

---

In [2]:
%load_ext autoreload
%autoreload 2

from umlcaxs_lib import votable_to_pandas, lognorm, mahalanobis, mahal_classifier_cl, create_summary_tables, mahal_classifier_all
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
import itertools
import seaborn as sns

from collections import Counter

%matplotlib inline

In [2]:
# Edit the font, font size, and axes width

mpl.rcParams['font.family'] = 'Avenir LT Std'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.linewidth'] = 2

In [3]:
uks = ['Star', 'X', 'Radio', 'IR', 'Blue', 'UV', 'gamma', 'PartofG', '**']
ltypes = ['QSO', 'AGN', 'Seyfert_1', 'Seyfert_2', 'HMXB', 'LMXB', 'XB', 'YSO', 'TTau*', 'Orion_V*']
grouped_replace = {'QSO': 'AGN', 'Seyfert_1': 'Seyfert', 'Seyfert_2': 'Seyfert', 'HMXB': 'XB', 'LMXB':'XB', 'TTau*':'YSO', 'Orion_V*': 'YSO' }

In [3]:
df_cluster_csc_simbad = pd.read_csv('./out_data/cluster_csc_simbad.csv', index_col=0)

/tmp/ipykernel_22809/1035316693.py:1: DtypeWarning: Columns (56,63,64,65,66,67,68,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cluster_csc_simbad = pd.read_csv('./out_data/cluster_csc_simbad.csv', index_col=0)


In [5]:
nans_df_csc_simbad = df_cluster_csc_simbad.copy(deep=True).fillna({'main_type': 'NaN'})

In [6]:
# Features that we use in our analysis
features = ['hard_hm', 'hard_hs', 'hard_ms', 'powlaw_gamma', 'bb_kt', 'var_prob_b','var_ratio_b', 'var_prob_h', 'var_ratio_h', 'var_prob_s', 'var_ratio_s', 'var_newq_b']

# Features to log transform and normalize
features_lognorm = ['bb_kt', 'var_ratio_h', 'var_ratio_b', 'var_ratio_s', 'var_newq_b']

# Features to normalize
features_norm = ['powlaw_gamma']

# Drop data with missing values in features
df_cscs_out = nans_df_csc_simbad.dropna(subset=features)

In [7]:
# Normalize or log normalize
df_cscs_lognorm = lognorm(df_cscs_out, features, features_norm, features_lognorm)

In [8]:
sum(df_cscs_lognorm.main_type.isin(uks+['NaN']))

14507

In [9]:
classified_df = mahal_classifier_all(df_cscs_lognorm, df_cscs_out, features, ltypes, uks=uks)
classified_df.head(10)

***Cluster 0***
***Cluster 1***
***Cluster 2***
***Cluster 3***
***Cluster 4***
***Cluster 5***


,name,obsid,cluster,main_type,QSO,AGN,Seyfert_1,Seyfert_2,HMXB,LMXB,...,R_simbad,J,H,K,u,g,r_simbada,i,z,angDist
0,2CXO J000010.0-501526,11997,1,QSO,7.109412e-01,1.998591e-01,3.817763e-02,1.198280e-03,2.087573e-02,4.708968e-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2CXO J000019.8-245030,13394,2,QSO,7.841575e-01,2.115698e-01,4.237418e-03,8.312877e-06,2.696935e-05,9.165581e-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2CXO J000025.4-245419,13394,1,Seyfert_1,4.459738e-05,1.858106e-04,6.535472e-01,3.462222e-01,3.457804e-08,1.228568e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2CXO J000027.4-500421,11742,1,AGN,1.823781e-04,8.045687e-01,6.197661e-02,3.051580e-10,1.188399e-01,1.034259e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2CXO J000027.4-500421,11997,1,Seyfert_1,4.029002e-02,4.302557e-03,8.071425e-01,1.109741e-05,4.020933e-04,2.280539e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2CXO J000028.5+623053,2810,3,AGN,2.199853e-03,7.341143e-01,1.090844e-05,2.907129e-07,4.578093e-03,2.590946e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2CXO J000031.1-500914,11997,0,Orion_V*,5.149786e-58,7.325791e-25,2.212079e-33,0.000000e+00,2.709021e-09,6.852283e-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2CXO J000031.8-245458,13394,5,QSO,4.335578e-01,6.251523e-03,1.896583e-03,8.076405e-06,3.305473e-01,5.331714e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2CXO J000047.6-551937,7061,2,QSO,7.616869e-01,5.975887e-02,1.785520e-01,2.229388e-06,2.421506e-09,1.382007e-32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2CXO J000058.9-245449,13394,1,QSO,6.777465e-01,2.986486e-01,2.052719e-02,3.545975e-05,2.002315e-04,7.983729e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
classified_df.to_csv('./out_data/classified_cscs.csv')

In [11]:
len(classified_df.name.unique())

8756

#### Create classification summary table
---

In [4]:
classified_df = pd.read_csv('./out_data/classified_cscs.csv', index_col=0)

In [7]:
counts = classified_df['name'].value_counts()

# Define bins for the grouping
bins = [1, 2, 3, 11, np.inf]
labels = ['1', '2', '3-10', '>10']

# Group the data based on the defined bins
grouped_counts = pd.cut(counts, bins=bins, labels=labels, right=False).value_counts()

# Sort the result by the index (the bins)
grouped_counts = grouped_counts.sort_index()

# Calculate the total number of sources
total_sources = grouped_counts.sum()

# Print the result with percentage
for range, num_sources in grouped_counts.items():
    percentage = (num_sources / total_sources) * 100
    print(f"{num_sources} sources ({percentage:.2f}%) have {range} detections.")

6802 sources (77.68%) have 1 detections.
1000 sources (11.42%) have 2 detections.
882 sources (10.07%) have 3-10 detections.
72 sources (0.82%) have >10 detections.


In [12]:
print(f'There are {len(np.unique(classified_df.name))} unique classified sources...')

There are 8756 unique classified sources...


In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 8))
ax.yaxis.set_tick_params(which='major', size=6, width=0.5, direction='out')
ax.yaxis.set_tick_params(which='minor', size=3, width=0.5, direction='in')
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(1000))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(250))
#ax.grid(True)
#sns.set_style("whitegrid")
sns.histplot(classified_df.name.value_counts().reset_index(name="counts").counts, color='black', bins=20, log_scale=True, ax=ax)
ax.set(xlabel='Count of detections', ylabel='Frequency')
ax.set_yscale('log')

plt.savefig('./figures/results_class_a_nofobs.pdf', bbox_inches='tight')  

In [13]:
def mean_std_round(x): return (str(round(np.mean(x), 3)) + '±' + str(round(np.std(x), 3)))
summ_table = classified_df.groupby('name')[ltypes].agg(mean_std_round)
summ_table['detection_count'] = classified_df.groupby(['name']).size()

summ_table_prov = classified_df.groupby('name')[ltypes].agg(['mean', 'std'])
class_mean_names = [list(tup) for tup in itertools.product(ltypes, ['mean'], repeat=1)]
names_comp = summ_table_prov[class_mean_names].idxmax(axis=1).to_list()
master_names = [name[0] for name in names_comp]
summ_table['soft_master_class'] = master_names
summ_table.sort_values('detection_count', ascending=False, inplace=True)

In [14]:
n_obs = 0
class_df_grouped = classified_df.copy(deep=True)
def most_common(x): 
    most_common,name_most_common = Counter(x).most_common(1)[0]
    return(most_common) 

summ_table_hard = class_df_grouped.groupby(['name']).size().to_frame(name='detection_count')
summ_table_hard = summ_table_hard[summ_table_hard.detection_count > n_obs]
summ_table_hard['hard_master_class'] = class_df_grouped.groupby(['name'])['main_type'].agg(most_common)
summ_table_hard.sort_values('detection_count', ascending=False, inplace=True)

In [15]:
agreeing_class_df = summ_table[summ_table.soft_master_class == summ_table_hard.hard_master_class].copy(deep=True)
agreeing_class_df = agreeing_class_df.rename(columns={'soft_master_class':'master_class'})
agreeing_class_df['agg_master_class'] = agreeing_class_df.master_class.replace(grouped_replace)

In [17]:
agreeing_class_df.shape

(8271, 13)

In [ ]:
cols = agreeing_class_df.columns.tolist()
first_cols = cols[-3:]
first_cols.reverse()
cols = first_cols + cols[:-3]
agreeing_class_df_out = agreeing_class_df[cols]

In [ ]:
agreeing_class_df_out.to_csv('./out_data/agreeing_classification.csv')

In [18]:
confused_class_df = summ_table[summ_table.soft_master_class != summ_table_hard.hard_master_class].copy(deep=True)
confused_class_df['hard_master_class'] = summ_table_hard.hard_master_class

In [20]:
confused_class_df.shape

(485, 13)

In [ ]:
cols = confused_class_df.columns.tolist()
first_cols = cols[-3:]
first_cols.reverse()
cols = first_cols + cols[:-3]
confused_class_df_out = confused_class_df[cols]

In [ ]:
confused_class_df_out.to_csv('./out_data/confused_classification.csv')